# EDA + REGRESSION + CLASSIFICATION FROM SCRATCH WITH BLACK FRIDAY DATASET

<img src='https://drive.google.com/uc?id=1ptXTg9tJspLaf6G3N7-rdUDMLeu7srM6' width=1000 >

<img src='' width=1000 >

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("input"))


['BlackFriday.csv']


In [3]:
# Importing the dataset
df = pd.read_csv('input/BlackFriday.csv')

In [4]:
# Let us check the Top 5 entries in the dataset.
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


# BASIC EDA

In [5]:
# Basic information 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 12 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 49.2+ MB


### 1). Dealing with Null values/ missing data.

In [6]:
# Let us check the Number of Null values and it's Percentage .
temp_df = df.isnull().sum().reset_index()
temp_df['Percentage of Null Values'] = temp_df[0]/len(df)*100
temp_df.columns = ['Column Name', 'Number of Null Values','Percentage of Null Values']
temp_df

,Column Name,Number of Null Values,Percentage of Null Values
0,User_ID,0,0.000000
1,Product_ID,0,0.000000
2,Gender,0,0.000000
3,Age,0,0.000000
4,Occupation,0,0.000000
5,City_Category,0,0.000000
6,Stay_In_Current_City_Years,0,0.000000
7,Marital_Status,0,0.000000
8,Product_Category_1,0,0.000000
9,Product_Category_2,166986,31.062713


* As there are 70% Null values in coulmn name Product_Category_3.
* So we will remove this column from our dataset.
* And Null values in column Product_Category_2, we will replaced it by mode of the column.

In [7]:
# Removing Product_Category_3 column
df.drop(labels='Product_Category_3', axis=1, inplace=True)

In [8]:
# Let us fill the Null values in Product_Categorical_2 column
# First let us check the product which come maximum number of times in this column.
new_df = df['Product_Category_2'].value_counts().reset_index()

new_df.iplot(kind='bar', x='index',y='Product_Category_2', title='Frequency of values in Product_Category_2 Column',
            xTitle='Product Category', yTitle='Frequency', color='deepskyblue')


* Now we will fill  the Null values with 8.
* One can also try not to fill the missing values.

In [9]:
# Filling Null values.
df['Product_Category_2'].fillna(value=8, inplace=True)
df['Product_Category_2'].unique()

array([ 8.,  6., 14.,  2., 15., 16., 11.,  5.,  3.,  4., 12.,  9., 10.,
       17., 13.,  7., 18.])

### ===========================================================================

### 2). Ratio of Male and Female.

In [10]:
gender = df['Gender'].value_counts().reset_index()
gender.columns = ['Gender', 'Count']
gender.iplot(kind='pie', labels='Gender', values='Count', hole=0.2, pull=0.2, title='Ratio of Male and Female')
# Interactive plot with plotly.

* 75% are Male as compare to 25% Female.
* Some Interesting Fact.
* It indicates that maximum number of products are releted to male or male are under pressure from thier wife to go to shopping :p .

### ======================================================================

### 3). From which age group people belongs maximum.

In [11]:
age = df['Age'].value_counts().reset_index()
age.columns = ['Age Group', 'Count']
age.iplot(kind='bar', x='Age Group', y='Count', title='Number of people belongs to different age group',
         xTitle='Age Group', yTitle='Quantity', color='deepskyblue')

#### Observation:-
* Maximum Number of people are from the Age group of 26-35.
* Young population is maximum.
* Children are in less number (0-17 age group).
* Old people are also there (55+ age group).

### =============================================================================

### 4). Let us first remove the unwanted columns.


In [12]:
try:
    df.drop(labels=['User_ID', 'Product_ID'], inplace=True, axis=1,)
except Exception as e: 
    print("Run from start again.")
df.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


### =======================================================================

### 5). Ratio of City Category.

In [13]:
city = df['City_Category'].value_counts().reset_index()
city.iplot(kind='pie', labels='index', values='City_Category', hole=0.2, pull=0.2,
          title='Percentage of People belongs to different City Category')

* Approximately equal number of people are there from city A,B and C.
* But from these all B city contains the maximum number of people.

### ==========================================================================

### 6). Let us see the total purchase amount of Male and Female.

In [14]:
temp_df = df.groupby('Gender', axis=0)['Purchase'].sum().reset_index()
temp_df.iplot(kind='bar', x='Gender', y='Purchase', title='Total Purchase amount of Male and Female',
             xTitle='Gender', yTitle='Total Purchase Amount', color='orange')


* As expected male have more total purchase amount than female due to large number of male as compare to female.
* Color orange indicates gender equality.

### =======================================================================

### 7). Ratio of married and non married poplutions.

In [15]:
status = df['Marital_Status'].value_counts().reset_index()
# Converting 0 and 1 into Married and Non Married in status DataFrame
status['index'] = status['index'].apply(lambda x: 'Non Married' if x==0 else 'Married')

status.iplot(kind='pie', labels='index', values='Marital_Status', hole=0.2, pull=0.2,
          title='Ratio of Married people and Non Married people')
status.T

,0,1
index,Non Married,Married
Marital_Status,317817,219760


#### Observations:-
* Non Married people come to maximum number of times to shoping.
* Non Married people are 59% and Married people are 41%.
* By this mall owner can increse the products which belongs to Non Married people.
* In this way owner can increase his/her profit.
* And the products which belongs to Non Married people, owner can place them near to entry gate or at a hot place of mall.

### ========================================================================

### 8). Total purchase amount of Married and Non Married people.

In [16]:
temp_df = df.groupby('Marital_Status', axis=0)['Purchase'].sum().reset_index()
temp_df['Marital_Status'] = temp_df['Marital_Status'].apply(lambda x: 'Non Married' if x==0 else 'Married')

temp_df.iplot(kind='bar', x='Marital_Status', y='Purchase', title='Total Purchase amount of Male and Female',
             xTitle='Marital_Status', yTitle='Total Purchase Amount', color='red')


* As expected, as Non Married people are more in number than Married people, so thier total purchase amount is also large.

### ========================================================================

### 9). Which Product Category Purchased maximum number of times.

In [17]:
pro1 = df['Product_Category_1'].reset_index()
pro1.columns = ['index', 'product']

pro2 = df['Product_Category_2'].reset_index()
pro2.columns = ['index', 'product']
product = pd.concat([pro1, pro2] )

product = product['product'].value_counts().reset_index()[:10]
product.iplot(kind='bar', x='index', y='product', title='Top 10 product number which purchase maximum number of times',
             xTitle='Product number', yTitle='Frequency', color='deepskyblue')

* So product number 8 is purchased maximum number of times.

### ======================================================================

### 10). Rich city from city category A,B,C.

In [18]:
rich = df.groupby('City_Category')['Purchase'].sum().reset_index()
rich.sort_values('Purchase',ascending=False, inplace=True)
rich.iplot(kind='pie', labels='City_Category' ,values='Purchase', hole=0.2, pull=0.2,
           title='Richest among A,B,C', )


* As one can B is the richest city among A,B,C with 41% followed by C and then followed by A.
* If mall owner knows that a particular person is from which city then worker there can show them products according to it, which lead to the benefit of the company.



### =========================================================================

### 11). Correlation between features on heatmap.

In [19]:
df.corr().iplot(kind='heatmap', title='HeatMap on Correlation of features ')

### ==========================================================================

# REGRESSION

In [20]:
# IMporting the useful Machine Learning libraries.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score


In [21]:
# Let us create a feature matrix and target vector.
x_train = df.iloc[:,:-1].values      # Feature matrix
y_train = df.iloc[:,-1].values     # Target vector

### 1). Dealing with categorical values.

In [22]:
# Apply LabelEncoder to convert categorical values into 0,1 and so on format.
label_x = LabelEncoder()
x_train[:,0] = label_x.fit_transform(x_train[:,0])
x_train[:,1] = label_x.fit_transform(x_train[:,1])
x_train[:,3] = label_x.fit_transform(x_train[:,3])
x_train[:,4] = label_x.fit_transform(x_train[:,4])

# Apply OneHotEncoder to split the columns which have more than 2 categories.
one_hot = OneHotEncoder(categorical_features=[1,2,3,4,6,7])
x_train = one_hot.fit_transform(x_train).toarray()
print(f"""Now let us see the shape of our x_train matrix.
Shape of x_train matrix =\t {x_train.shape}.
So now we have 73 columns.
Now we will apply Dimensionalty Reduction to reduce the number of features/columns with the help of PCA algorithm. """)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning:

The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.



Now let us see the shape of our x_train matrix.
Shape of x_train matrix =	 (537577, 73).
So now we have 73 columns.
Now we will apply Dimensionalty Reduction to reduce the number of features/columns with the help of PCA algorithm. 


### ================================================================


### 2). Scaling the feature matrix, so that we can compare them on the same scale.

In [23]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)

### 3). Dimensionalty Reduction with the help of PCA algorithm.

In [24]:
pca = PCA(n_components=None)
x_train = pca.fit_transform(x_train)
pca.explained_variance_ratio_

array([3.07961958e-02, 2.80003491e-02, 2.66756054e-02, 2.54718396e-02,
       2.33202625e-02, 2.25407747e-02, 2.13145111e-02, 2.10317684e-02,
       2.04429437e-02, 2.00029285e-02, 1.89966429e-02, 1.87807791e-02,
       1.84106864e-02, 1.81799034e-02, 1.75751546e-02, 1.74209028e-02,
       1.72303758e-02, 1.70204837e-02, 1.68322704e-02, 1.64045044e-02,
       1.63145089e-02, 1.58948567e-02, 1.57087607e-02, 1.51043525e-02,
       1.49086623e-02, 1.48346772e-02, 1.46003913e-02, 1.45148249e-02,
       1.44051786e-02, 1.43241345e-02, 1.42060894e-02, 1.41638065e-02,
       1.41516376e-02, 1.41276351e-02, 1.40778659e-02, 1.40248895e-02,
       1.39768981e-02, 1.39594325e-02, 1.39078744e-02, 1.38717463e-02,
       1.37867399e-02, 1.37354140e-02, 1.37154520e-02, 1.37061044e-02,
       1.36595277e-02, 1.36092734e-02, 1.35638587e-02, 1.34920774e-02,
       1.33647545e-02, 1.32708359e-02, 1.31506711e-02, 1.28339284e-02,
       1.26882779e-02, 1.24630729e-02, 1.18506095e-02, 1.07824242e-02,
      

* So now we will take 50 features (taking number of features with e-02). 

In [25]:
pca = PCA(n_components=50)   # Fitting PCA with 50 features.
x_train = pca.fit_transform(x_train)

### ===========================================================================

### 4). Apply Multi-linear Regression and Support vector regressor Model.

#### a). Mutli-linear Regression Model.

In [26]:
# Multi-linear regression Model. 
regressor_multi = LinearRegression()
regressor_multi.fit(x_train,y_train)
# Let us check the accuray
accuracy = cross_val_score(estimator=regressor_multi, X=x_train, y=y_train,cv=10)
print(f"The accuracy of the Multi-linear Regressor Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")
print()

The accuracy of the Multi-linear Regressor Model is 	 0.4826316291035549
The deviation in the accuracy is 	 0.0018466754589391726



### Observation:-
* Here we are getting accuracy of 50% with deviation of only 0.2%.
* In my view this is good accuracy with this deviation.
* Here we are getting low accuracy, it may indicates that our model may be not linear, so by applying the other non-linear algothim we may get more accuracy.

#### b). SVR Model

In [27]:
"""
# SVR 
regressor_svr = SVR(kernel='rbf')
regressor_svr.fit(x_train, y_train)
# Let us check the accuracy
accuracy = cross_val_score(estimator=regressor_svr, X=x_train, y=y_train,cv=10)
print(f"The accuracy of the SVR Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")
print()
"""
print("Apply this model, if accuracy is good then use this model")
print("This model takes too much time. ")

Apply this model, if accuracy is good then use this model
This model takes too much time. 


### =============================================================================

# CLASSIFICATION

* Here On the basis of data we will classify whether a person is married or not.

In [28]:
df.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [29]:
# Creating Feature matrix and Target vector.
x_train = df.drop(labels='Marital_Status',axis=1).values
y_train = df['Marital_Status'].values

### 1). Dealing with categorical values.

In [30]:
# Apply LabelEncoder to convert categorical values into 0,1 and so on format.
label_x = LabelEncoder()
x_train[:,0] = label_x.fit_transform(x_train[:,0])
x_train[:,1] = label_x.fit_transform(x_train[:,1])
x_train[:,3] = label_x.fit_transform(x_train[:,3])
x_train[:,4] = label_x.fit_transform(x_train[:,4])

# Apply OneHotEncoder to split the columns which have more than 2 categories.
one_hot = OneHotEncoder(categorical_features=[1,2,3,4,5,6])
x_train = one_hot.fit_transform(x_train).toarray()
print(f"""Now let us see the shape of our x_train matrix.
Shape of x_train matrix =\t {x_train.shape}.
So now we have 73 columns.
Now we will apply Dimensionalty Reduction to reduce the number of features/columns with the help of PCA algorithm. """)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning:

The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.



Now let us see the shape of our x_train matrix.
Shape of x_train matrix =	 (537577, 73).
So now we have 73 columns.
Now we will apply Dimensionalty Reduction to reduce the number of features/columns with the help of PCA algorithm. 


### ==============================================================

### 2). Scaling the feature matric.

In [31]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)

### 3). Dimensionality Reduction with Principle Component Analysis (PCA).

In [32]:
pca = PCA(n_components=None)
x_train = pca.fit_transform(x_train)
pca.explained_variance_ratio_

array([3.53357058e-02, 2.80498398e-02, 2.61408396e-02, 2.49312906e-02,
       2.33543900e-02, 2.27666734e-02, 2.23418118e-02, 2.12534619e-02,
       2.07610391e-02, 2.00420822e-02, 1.91924455e-02, 1.89227796e-02,
       1.85755249e-02, 1.84077865e-02, 1.77527490e-02, 1.73747768e-02,
       1.70878778e-02, 1.69484756e-02, 1.68005614e-02, 1.63968409e-02,
       1.61216774e-02, 1.60855880e-02, 1.56540311e-02, 1.51804275e-02,
       1.50041846e-02, 1.48461791e-02, 1.46743349e-02, 1.45722772e-02,
       1.44193786e-02, 1.43185505e-02, 1.42299564e-02, 1.41864001e-02,
       1.41473868e-02, 1.40961591e-02, 1.40547547e-02, 1.40329313e-02,
       1.40052995e-02, 1.39745334e-02, 1.39468153e-02, 1.38915786e-02,
       1.37979662e-02, 1.37295287e-02, 1.37117865e-02, 1.36746374e-02,
       1.36612442e-02, 1.36099818e-02, 1.35554224e-02, 1.34938759e-02,
       1.33488388e-02, 1.32355347e-02, 1.31499472e-02, 1.27951226e-02,
       1.26870883e-02, 1.25003039e-02, 1.19051941e-02, 1.09951523e-02,
      

In [33]:
pca = PCA(n_components=50)   # Fitting PCA with 50 features.
x_train = pca.fit_transform(x_train)

### ===========================================================================

### 4). Apply Logistic Regression Model.

In [34]:
# Apply Logistic regression
# First step is to train our model .

classifier_logi = LogisticRegression()
classifier_logi.fit(x_train,y_train)

# Let us check the accuracy of the model
accuracy = cross_val_score(estimator=classifier_logi, X=x_train, y=y_train, cv=10)
print(f"The accuracy of the Logistic Regressor Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")

The accuracy of the Logistic Regressor Model is 	 0.658878629350484
The deviation in the accuracy is 	 0.0031776294900038716


### Observations:-
* Accuracy of 65% with only 0.2% deviation.
* Our model is good.
* Now we can also apply other model, to check the accuray of other models.
* But other models will take so much time to train.
* I am here providing the code of SVC Model and Navie-Bayes Model, but I am not ruuning them right now.
* Feel free to use code of these models to train your model

### 5). Apply Support Vector Classifier (SVC).

In [35]:
"""

# Apply SVM with Gaussian kernel
classifier_svm2 = SVC(kernel='rbf', )
classifier_svm2.fit(x_train,y_train)
accuracy = cross_val_score(estimator=classifier_svm2, X=x_train, y=y_train, cv=10)
print(f"The accuracy of the SVM Gaussian kernel Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")
"""
print("SVC Model")

SVC Model


### 6). Naiye-Bayes Model.

In [36]:
"""

# Apply Naive Bayes Model.
# Train Model
classifier_bayes = GaussianNB()
classifier_bayes.fit(x_train,y_train)
# Check the accuracy and deviation in the accuracy
accuracy = cross_val_score(estimator=classifier_bayes, X=x_train, y=y_train, cv=10)
print(f"The accuracy of the Naive Bayes Model is \t {accuracy.mean()}") 
print(f"The deviation in the accuracy is \t {accuracy.std()}")
"""
print("Navie-Bayes Model")

Navie-Bayes Model


* Likewise one can also predict the Gender of a person.

### ======================================================================